In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import statsmodels.api as sm
import seaborn as sns
import matplotlib.pyplot as plt
from patsy import dmatrix
from sklearn.metrics import mean_squared_error
from pyearth import Earth
from sklearn.metrics import roc_curve, precision_recall_curve, auc, make_scorer, recall_score, accuracy_score, precision_score, confusion_matrix
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_predict, cross_val_score, cross_validate
from patsy import dmatrix
import statsmodels.formula.api as smf
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import accuracy_score, mean_absolute_error, mean_squared_error, r2_score, roc_curve, auc, precision_score, recall_score, confusion_matrix

from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV, ParameterGrid
from sklearn.ensemble import BaggingRegressor,BaggingClassifier,RandomForestClassifier, RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import SCORERS

#Libraries for visualizing trees
from sklearn.tree import export_graphviz
from six import StringIO
from IPython.display import Image

#import pydotplus
import time as tm

import itertools as it

In [2]:
train = pd.read_csv('train_final.csv')
test = pd.read_csv('test.csv')

In [3]:
train_final = pd.get_dummies(train)
y = train_final.Credit_Rating
X = train_final.drop(columns = 'Credit_Rating')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 45)

X = X_train[X_train.columns]
y = y_train

## Grid Search

In [4]:
n_samples = train.shape[0]
n_features = train.shape[1]

params = {'base_estimator': [DecisionTreeRegressor(random_state = 1),LinearRegression()],#Comparing bagging with a linear regression model as well
          'n_estimators': [150,200,250],
          'max_samples': [0.5,1.0],
          'max_features': [0.5,1.0],
          'bootstrap': [True, False],
          'bootstrap_features': [True, False]}

cv = KFold(n_splits=5,shuffle=True,random_state=1)
bagging_regressor_grid = GridSearchCV(BaggingRegressor(random_state=1, n_jobs=-1), 
                                      param_grid =params, cv=cv, n_jobs=-1, verbose=1)
bagging_regressor_grid.fit(X, y)

Fitting 5 folds for each of 96 candidates, totalling 480 fits


GridSearchCV(cv=KFold(n_splits=5, random_state=1, shuffle=True),
             estimator=BaggingRegressor(n_jobs=-1, random_state=1), n_jobs=-1,
             param_grid={'base_estimator': [DecisionTreeRegressor(random_state=1),
                                            LinearRegression()],
                         'bootstrap': [True, False],
                         'bootstrap_features': [True, False],
                         'max_features': [0.5, 1.0], 'max_samples': [0.5, 1.0],
                         'n_estimators': [150, 200, 250]},
             verbose=1)

In [5]:
print('Best Parameters : ',bagging_regressor_grid.best_params_)

Best Parameters :  {'base_estimator': DecisionTreeRegressor(random_state=1), 'bootstrap': True, 'bootstrap_features': True, 'max_features': 1.0, 'max_samples': 1.0, 'n_estimators': 250}


In [6]:
#Model with optimal parameters
model = BaggingRegressor(base_estimator=DecisionTreeRegressor(), n_estimators=250, random_state=1,
                        oob_score=True,n_jobs=-1,bootstrap_features=True,bootstrap=True,
                        max_features=1.0,max_samples=1.0).fit(X, y)

#RMSE on test data
RMSE = np.sqrt(mean_squared_error(y_test, model.predict(X_test)))
print(RMSE)


61.619785272969395


# Finer Grid Search 1

In [7]:
n_samples = train.shape[0]
n_features = train.shape[1]

finer_params1 = {'base_estimator': [DecisionTreeRegressor(random_state = 1),LinearRegression()],#Comparing bagging with a linear regression model as well
          'n_estimators': [250, 300, 350],
          'max_samples': [1.0, 1.5],
          'max_features': [1.0, 1.5],
          'bootstrap': [True],
          'bootstrap_features': [True]}

cv = KFold(n_splits=5,shuffle=True,random_state=1)
bagging_regressor_grid_finer1 = GridSearchCV(BaggingRegressor(random_state=1, n_jobs=-1), 
                                      param_grid =finer_params1, cv=cv, n_jobs=-1, verbose=1)
bagging_regressor_grid_finer1.fit(X, y)

print('Best Parameters : ',bagging_regressor_grid_finer1.best_params_)

Fitting 5 folds for each of 24 candidates, totalling 120 fits
Best Parameters :  {'base_estimator': DecisionTreeRegressor(random_state=1), 'bootstrap': True, 'bootstrap_features': True, 'max_features': 1.0, 'max_samples': 1.0, 'n_estimators': 300}


In [8]:
#Model with optimal parameters
model_finer1 = BaggingRegressor(base_estimator=DecisionTreeRegressor(), n_estimators=250, random_state=1,
                        oob_score=True,n_jobs=-1,bootstrap_features=True,bootstrap=True,
                        max_features=1.0,max_samples=1.0).fit(X, y)

#RMSE on test data
RMSE = np.sqrt(mean_squared_error(y_test, model_finer1.predict(X_test)))
print(RMSE)


61.619785272969395


# Finer Grid Search 2

In [9]:
n_samples = train.shape[0]
n_features = train.shape[1]

finer_params2 = {'base_estimator': [DecisionTreeRegressor(random_state = 1),LinearRegression()],#Comparing bagging with a linear regression model as well
          'n_estimators': [250, 300, 350],
          'max_samples': [.75, 1.0, 1.25],
          'max_features': [.75, 1.0, 1.25],
          'bootstrap': [True],
          'bootstrap_features': [True]}

cv = KFold(n_splits=5,shuffle=True,random_state=1)
bagging_regressor_grid_finer2 = GridSearchCV(BaggingRegressor(random_state=1, n_jobs=-1), 
                                      param_grid =finer_params2, cv=cv, n_jobs=-1, verbose=1)
bagging_regressor_grid_finer2.fit(X_train, y_train)

print('Best Parameters : ',bagging_regressor_grid_finer1.best_params_)

Fitting 5 folds for each of 54 candidates, totalling 270 fits
Best Parameters :  {'base_estimator': DecisionTreeRegressor(random_state=1), 'bootstrap': True, 'bootstrap_features': True, 'max_features': 1.0, 'max_samples': 1.0, 'n_estimators': 300}


In [11]:
#Model with optimal parameters
model_finer2 = BaggingRegressor(base_estimator=DecisionTreeRegressor(), n_estimators=300, random_state=1,
                        oob_score=True,n_jobs=-1,bootstrap_features=True,bootstrap=True,
                        max_features=1.0,max_samples=1.0).fit(X, y)

#RMSE on test data
RMSE = np.sqrt(mean_squared_error(y_test, model_finer2.predict(X_test)))
print(RMSE)


61.57320248750542
